In [112]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import grangercausalitytests
import warnings
from sklearn.tree import DecisionTreeRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
from sklearn.metrics import mean_squared_error
import warnings
from dateutil.relativedelta import relativedelta
from datetime import datetime
warnings.filterwarnings("ignore")
%matplotlib inline
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
df = pd.read_excel('Dataset_palm_oil_forecasting.xlsx')
ElNino = pd.read_excel('Elnino_Indicator.xlsx')

In [92]:
df = df.merge(ElNino[['Date', 'ElNinoIndicator']], on='Date', how='left')

In [93]:
df.head()

In [94]:
df.columns

In [95]:
df_regions = df.drop(['Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Temp_min_weighted_avg',
       'Temp_max_weighted_avg', 'Wind_speed_weighted_avg', 'Temp_avg_weighted_avg', 'Days_temp_below_weighted_avg', 'Days_temp_above_weighted_avg',
       'Amplitude_weighted_avg'], axis = 1)
df_avg = df[['Date', 'Palm_Oil', 'Potash_Qty_kg/ha','Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Temp_min_weighted_avg',
       'Temp_max_weighted_avg', 'Wind_speed_weighted_avg', 'Temp_avg_weighted_avg', 'Days_temp_below_weighted_avg', 'Days_temp_above_weighted_avg',
       'Amplitude_weighted_avg', 'Sunshine_weighted_avg', 'ElNinoIndicator']]

In [96]:
df_avg.describe(include=[np.number])

In [97]:
df_regions.describe(include=[np.number])

In [98]:
# List of regions and variables for line chart comparison
regions = ["JOHOR", "PAHANG", "SABAH", "SARAWAK"]
variables = ["Evapo", "Precip_sum", "Temp_avg", "Temp_min", "Temp_max", "Wind_speed", "Amplitude", "Days_temp_above", "Days_temp_below", "Sunshine"]

# Loop through the variables to create line charts for each
for variable in variables:
    plt.figure(figsize=(12, 6))
    for region in regions:
        region_col = f"{variable}_{region}"
        if region_col in df_regions.columns:
            plt.plot(df_regions['Date'], df_regions[region_col], label=region)
    
    # Add chart details
    plt.title(f"{variable} Trends Across Regions", fontsize=16)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel(variable.replace("_", " "), fontsize=12)
    plt.legend(title="Region", fontsize=10)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [99]:
plt.figure(figsize=(12, 6))
plt.plot(df_avg['Date'], df_avg['Potash_Qty_kg/ha'])

In [100]:
plt.figure(figsize=(12, 6))
plt.plot(df_avg['Date'], df_avg['ElNinoIndicator'])

## Features elimination 

#### Correlation heatmap

First step is to check correlation heatmap. We can eliminate one of two features where correlation is highly positive or negative. 

In [101]:
plt.figure(figsize=(10, 6))
sns.heatmap(df_avg.corr(), annot=True, fmt=".2f", cmap = "RdYlGn", cbar=True)

##### Based on above heatmap we can eliminate:
- **Precipitation** - it's negatively correlated with evapotranspiration (logically makes sense), we don't need to keep both of them
- **Min, Max, Avg temperature** - we have other columns related to temperature like days above and below threshold, aplitude

In [102]:
df_corr = df_avg.drop(['Precip_sum_weighted_avg', 'Temp_min_weighted_avg', 'Temp_max_weighted_avg', 'Temp_avg_weighted_avg'], axis = 1)
df_corr = df_corr.dropna()
plt.figure(figsize=(10, 6))
sns.heatmap(df_corr.corr(), annot=True, fmt=".2f", cmap = "RdYlGn", cbar=True)

In [103]:
df_corr = df_avg.drop(['Precip_sum_weighted_avg','Evapo_weighted_avg', 'Temp_min_weighted_avg', 'Temp_max_weighted_avg', 'Temp_avg_weighted_avg'], axis = 1)
df_corr = df_corr.dropna()
plt.figure(figsize=(10, 6))
sns.heatmap(df_corr.corr(), annot=True, fmt=".2f", cmap = "RdYlGn", cbar=True)

In [104]:
df_corr1 = df_avg[['Palm_Oil','Potash_Qty_kg/ha', 
       'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Evapo_weighted_avg', 'ElNinoIndicator']]
df_corr1 = df_corr1.dropna()
plt.figure(figsize=(10, 6))
sns.heatmap(df_corr1.corr(), annot=True, fmt=".2f", cmap = "RdYlGn", cbar=True)

#### VIF

**Interpreting VIF Values**
- VIF = 1: No correlation (ideal)
- VIF < 5: Low multicollinearity (acceptable)
- VIF 5-10: Moderate multicollinearity (consider removing)
- VIF > 10: High multicollinearity (remove or combine features)

In [105]:
X = df[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Wind_speed_weighted_avg', 'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg', 'ElNinoIndicator']]
Y = df[['Palm_Oil']]

if 'constant' in X.columns:
    X = X.drop(columns=['constant'])

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

print(vif_data)

In [106]:
X = df[['Potash_Qty_kg/ha',
       'Wind_speed_weighted_avg', 
        'Amplitude_weighted_avg',
       'Sunshine_weighted_avg', 'ElNinoIndicator']]
Y = df[['Palm_Oil']]

if 'constant' in X.columns:
    X = X.drop(columns=['constant'])

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

print(vif_data)

###### After removing features with VIF value above 5 one by one. 

In [107]:
df_avg = df_avg.dropna()
X = df_avg[['Potash_Qty_kg/ha', 
       'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Evapo_weighted_avg', 'ElNinoIndicator']]
Y = df_avg[['Palm_Oil']]

if 'constant' in X.columns:
    X = X.drop(columns=['constant'])

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

print(vif_data)

In [108]:
df_corr2 = df_avg[['Palm_Oil', 'Potash_Qty_kg/ha',
       'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'ElNinoIndicator']]
df_corr2 = df_corr2.dropna()
plt.figure(figsize=(10, 6))
sns.heatmap(df_corr2.corr(), annot=True, fmt=".2f", cmap = "RdYlGn", cbar=True)

###### Building OLS regression model based on features choosen by VIF.

In [109]:
X = df_avg[['Potash_Qty_kg/ha', 
       'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Precip_sum_weighted_avg']]
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

In [110]:
#df_avg['Date'] = pd.to_datetime(df_avg['Date'])
#df_avg.set_index('Date', inplace=True)

#### Checking lags for all variables

In [88]:
# Define maximum lag
max_lag = 12

# Generate lagged features for all variables
lagged_df = pd.DataFrame(index=df_avg.index)
for col in df_avg.columns:
    for lag in range(0, max_lag + 1):
        lagged_df[f"{col}_lag{lag}"] = df_avg[col].shift(lag)

# Add original Palm_Oil column to lagged_df for correlation calculations
lagged_df["Palm_Oil"] = df_avg["Palm_Oil"]

# Create heatmaps for 12 lags of Palm_Oil vs 12 lags of each variable
for variable in df_avg.columns:
    # Generate lagged column names for Palm_Oil and the current variable
    palm_oil_lags = [f"Palm_Oil_lag{lag}" for lag in range(0, max_lag + 1)]
    variable_lags = [f"{variable}_lag{lag}" for lag in range(0, max_lag + 1)]

    # Filter existing columns in lagged_df
    palm_oil_lags = [col for col in palm_oil_lags if col in lagged_df.columns]
    variable_lags = [col for col in variable_lags if col in lagged_df.columns]

    # Compute correlation matrix
    corr_matrix = lagged_df[palm_oil_lags].corrwith(lagged_df[variable_lags[0]])
    for lag_col in variable_lags[1:]:
        col_corr = lagged_df[palm_oil_lags].corrwith(lagged_df[lag_col])
        corr_matrix = pd.concat([corr_matrix, col_corr], axis=1)

    # Convert to DataFrame for heatmap
    corr_matrix = pd.DataFrame(corr_matrix)
    corr_matrix.columns = variable_lags
    corr_matrix.index = palm_oil_lags

    # Plot heatmap with annotations
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap = "RdYlGn", fmt=".2f", cbar=True)
    plt.title(f"Palm_Oil Lags vs {variable} Lags")
    plt.xlabel("Lags of Variable")
    plt.ylabel("Lags of Palm_Oil")
    plt.show()

In [ ]:
etsdecompos = seasonal_decompose(df_avg['Palm_Oil'])
rcParams['figure.figsize']=(12,6)
etsdecompos.plot();

In [ ]:
for variable in df_avg.drop(['Potash_Qty_kg/ha'], axis = 1).columns:

    # Perform the ADF test
    result = adfuller(df_avg.drop(['Potash_Qty_kg/ha'], axis = 1)[variable])

    # Extract and print the p-value from the test result
    p_value = result[1]
    print("p-value:", p_value)

    # Interpret the result
    if p_value <= 0.05:
        print(f"The variable {variable} is stationary.\n")
    else:
        print(f"The variable {variable} is not stationary.\n")

In [ ]:
df_avg.columns

In [ ]:
df_reg = df_avg.copy()
df_reg = df_reg.dropna()

In [ ]:
X = df_reg[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Temp_min_weighted_avg',
       'Temp_max_weighted_avg', 'Wind_speed_weighted_avg',
       'Temp_avg_weighted_avg', 'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
Y = df_reg['Palm_Oil']

In [ ]:
# Ensure no constant column (like an intercept)
if 'constant' in X.columns:
    X = X.drop(columns=['constant'])

# Create an empty DataFrame for VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

# Display VIF values
print(vif_data)

In [ ]:
x1 = df_reg[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 
       'Temp_max_weighted_avg', 'Wind_speed_weighted_avg',
       'Temp_avg_weighted_avg', 'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
# Ensure no constant column (like an intercept)
if 'constant' in x1.columns:
    x1 = x1.drop(columns=['constant'])

# Create an empty DataFrame for VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = x1.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(x1.values, i) for i in range(x1.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

# Display VIF values
print(vif_data)

In [ ]:
x2 = df_reg[['Potash_Qty_kg/ha', 
       'Precip_sum_weighted_avg', 'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg']]

if 'constant' in x2.columns:
    x2 = x2.drop(columns=['constant'])

vif_data = pd.DataFrame()
vif_data["Feature"] = x2.columns

vif_data["VIF"] = [variance_inflation_factor(x2.values, i) for i in range(x2.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

print(vif_data)

In [ ]:
x2 = sm.add_constant(x2)

model = sm.OLS(Y, x2).fit()

print(model.summary())

In [ ]:
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

In [ ]:
X2 = df_reg[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Temp_min_weighted_avg', 'Wind_speed_weighted_avg',
       'Temp_avg_weighted_avg', 'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
X2 = sm.add_constant(X2)
model = sm.OLS(Y, X2).fit()
print(model.summary())

In [ ]:
X3 = df_reg[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Wind_speed_weighted_avg',
       'Temp_avg_weighted_avg', 'Days_temp_below_weighted_avg',
       'Days_temp_above_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
X3 = sm.add_constant(X3)
model = sm.OLS(Y, X3).fit()
print(model.summary())

In [ ]:
X4 = df_reg[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Wind_speed_weighted_avg',
       'Temp_avg_weighted_avg', 'Days_temp_below_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
X4 = sm.add_constant(X4)
model = sm.OLS(Y, X4).fit()
print(model.summary())

In [ ]:
X5 = df_reg[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Wind_speed_weighted_avg',
       'Days_temp_below_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
X5 = sm.add_constant(X5)
model = sm.OLS(Y, X5).fit()
print(model.summary())

In [ ]:
X6 = df_reg[['Potash_Qty_kg/ha', 'Evapo_weighted_avg',
       'Precip_sum_weighted_avg', 'Wind_speed_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
X6 = sm.add_constant(X6)
model = sm.OLS(Y, X6).fit()
print(model.summary())

In [ ]:
X7 = df_reg[['Potash_Qty_kg/ha',
       'Precip_sum_weighted_avg', 'Wind_speed_weighted_avg', 'Amplitude_weighted_avg',
       'Sunshine_weighted_avg']]
X7 = sm.add_constant(X7)
model = sm.OLS(Y, X7).fit()
print(model.summary())

In [ ]:
if 'constant' in X7.columns:
    X7 = X7.drop(columns=['constant'])

vif_data = pd.DataFrame()
vif_data["Feature"] = X7.columns

vif_data["VIF"] = [variance_inflation_factor(X7.values, i) for i in range(X7.shape[1])]

print(vif_data)

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
# Calculate 3-month moving average for all numeric columns
df_3M_avg = df.rolling(window=3).mean().add_suffix('_3MA')
df_3M_avg

In [ ]:
df_3M_avg = df_3M_avg.dropna()
df_3M_avg

In [ ]:
df_3M_avg.columns

In [ ]:
# Define maximum lag
max_lag = 12

# Generate lagged features for all variables
lagged_df = pd.DataFrame(index=df_3M_avg.index)
for col in df_3M_avg.columns:
    for lag in range(0, max_lag + 1):
        lagged_df[f"{col}_lag{lag}"] = df_3M_avg[col].shift(lag)

# Add original Palm_Oil column to lagged_df for correlation calculations
lagged_df["Palm_Oil_3MA"] = df_3M_avg["Palm_Oil_3MA"]

# Create heatmaps for 12 lags of Palm_Oil vs 12 lags of each variable
for variable in df_3M_avg.columns:
    # Generate lagged column names for Palm_Oil and the current variable
    palm_oil_lags = [f"Palm_Oil_3MA_lag{lag}" for lag in range(0, max_lag + 1)]
    variable_lags = [f"{variable}_lag{lag}" for lag in range(0, max_lag + 1)]

    # Filter existing columns in lagged_df
    palm_oil_lags = [col for col in palm_oil_lags if col in lagged_df.columns]
    variable_lags = [col for col in variable_lags if col in lagged_df.columns]
    
    # Compute correlation matrix
    corr_matrix = lagged_df[palm_oil_lags].corrwith(lagged_df[variable_lags[0]])
    for lag_col in variable_lags[1:]:
        col_corr = lagged_df[palm_oil_lags].corrwith(lagged_df[lag_col])
        corr_matrix = pd.concat([corr_matrix, col_corr], axis=1)

    # Convert to DataFrame for heatmap
    corr_matrix = pd.DataFrame(corr_matrix)
    corr_matrix.columns = variable_lags
    corr_matrix.index = palm_oil_lags
    corr_matrix
    # Plot heatmap with annotations
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap = "RdYlGn", fmt=".2f", cbar=True)
    plt.title(f"Palm_Oil Lags vs {variable} Lags")
    plt.xlabel("Lags of Variable")
    plt.ylabel("Lags of Palm_Oil")
    plt.show()

In [ ]:
df_3M_avg.columns

In [ ]:
X = df_3M_avg[['Potash_Qty_kg/ha_3MA', 'Evapo_weighted_avg_3MA',
       'Precip_sum_weighted_avg_3MA', 
       'Temp_max_weighted_avg_3MA', 'Wind_speed_weighted_avg_3MA',
       'Temp_avg_weighted_avg_3MA', 'Days_temp_below_weighted_avg_3MA',
       'Days_temp_above_weighted_avg_3MA', 'Amplitude_weighted_avg_3MA',
       'Sunshine_weighted_avg_3MA']]
# Ensure no constant column (like an intercept)
if 'constant' in X.columns:
    X = X.drop(columns=['constant'])

# Create an empty DataFrame for VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

# Display VIF values
print(vif_data)

In [ ]:
X = df_3M_avg[['Potash_Qty_kg/ha_3MA', 'Evapo_weighted_avg_3MA',
       'Days_temp_below_weighted_avg_3MA',
       'Days_temp_above_weighted_avg_3MA']]
Y = df_3M_avg['Palm_Oil_3MA']
# Ensure no constant column (like an intercept)
if 'constant' in X.columns:
    X = X.drop(columns=['constant'])

# Create an empty DataFrame for VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data["VIF"] = vif_data["VIF"].apply(lambda x: f"{x:.2f}")

# Display VIF values
print(vif_data)

In [ ]:
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

In [ ]:
X3 = df_3M_avg[['Potash_Qty_kg/ha_3MA', 'Evapo_weighted_avg_3MA', 'Precip_sum_weighted_avg_3MA',
       'Temp_min_weighted_avg_3MA', 'Temp_max_weighted_avg_3MA',
       'Wind_speed_weighted_avg_3MA', 'Sunshine_weighted_avg_3MA',
       'Temp_avg_weighted_avg_3MA', 'Days_temp_below_weighted_avg_3MA',
       'Days_temp_above_weighted_avg_3MA', 'Amplitude_weighted_avg_3MA']]
Y3 = df_3M_avg['Palm_Oil_3MA']
X3 = sm.add_constant(X3)
model = sm.OLS(Y3, X3).fit()
print(model.summary())

In [ ]:
X4 = df_3M_avg[['Potash_Qty_kg/ha_3MA', 'Evapo_weighted_avg_3MA', 'Precip_sum_weighted_avg_3MA',
       'Temp_min_weighted_avg_3MA', 
       'Wind_speed_weighted_avg_3MA', 'Sunshine_weighted_avg_3MA',
       'Temp_avg_weighted_avg_3MA', 'Days_temp_below_weighted_avg_3MA',
       'Days_temp_above_weighted_avg_3MA', 'Amplitude_weighted_avg_3MA']]
Y4 = df_3M_avg['Palm_Oil_3MA']
X4 = sm.add_constant(X4)
model = sm.OLS(Y4, X4).fit()
print(model.summary())

In [ ]:
X5 = df_3M_avg[['Potash_Qty_kg/ha_3MA', 'Evapo_weighted_avg_3MA', 'Precip_sum_weighted_avg_3MA',
       'Wind_speed_weighted_avg_3MA', 'Sunshine_weighted_avg_3MA',
       'Temp_avg_weighted_avg_3MA', 'Days_temp_below_weighted_avg_3MA',
       'Days_temp_above_weighted_avg_3MA', 'Amplitude_weighted_avg_3MA']]
Y5 = df_3M_avg['Palm_Oil_3MA']
X5 = sm.add_constant(X5)
model = sm.OLS(Y5, X5).fit()
print(model.summary())